In [2]:
%load_ext rpy2.ipython

In [ ]:
%%R
library(readxl)
data=read.csv("/content/NSSO68 (1).csv")

In [ ]:
%%R
#Load required libraries
library(dplyr)

In [11]:
%%R
# Filtering for Gujarat
df=data %>%
  filter(state_1 == "GUJ")

In [12]:
%%R
# Display dataset info
cat("Dataset Information:\n")
print(names(df))
print(head(df))
print(dim(df))

Dataset Information:
  [1] "slno"                            "grp"                            
  [3] "Round_Centre"                    "FSU_number"                     
  [5] "Round"                           "Schedule_Number"                
  [7] "Sample"                          "Sector"                         
  [9] "state"                           "State_Region"                   
 [11] "District"                        "Stratum_Number"                 
 [13] "Sub_Stratum"                     "Schedule_type"                  
 [15] "Sub_Round"                       "Sub_Sample"                     
 [17] "FOD_Sub_Region"                  "Hamlet_Group_Sub_Block"         
 [19] "t"                               "X_Stage_Stratum"                
 [21] "HHS_No"                          "Level"                          
 [23] "Filler"                          "hhdsz"                          
 [25] "NIC_2008"                        "NCO_2004"                       
 [27] "HH_type"  

In [13]:
%%R
#Finding missing values
missing_info=colSums(is.na(df))
cat("Missing Values Information:\n")
print(missing_info)

Missing Values Information:
                           slno                             grp 
                              0                               0 
                   Round_Centre                      FSU_number 
                              0                               0 
                          Round                 Schedule_Number 
                              0                               0 
                         Sample                          Sector 
                              0                               0 
                          state                    State_Region 
                              0                               0 
                       District                  Stratum_Number 
                              0                               0 
                    Sub_Stratum                   Schedule_type 
                              0                               0 
                      Sub_Round                      Sub_Sampl

In [22]:
%%R
# Sub-setting the data
gujnew=df %>%
  select(state_1, District, Region, Sector, State_Region, Meals_At_Home, ricepds_v, Wheatpds_q, chicken_q, pulsep_q, wheatos_q, No_of_Meals_per_day)


In [23]:
%%R
# Check for missing values in the subset
cat("Missing Values in Subset:\n")
print(colSums(is.na(gujnew)))

Missing Values in Subset:
            state_1            District              Region              Sector 
                  0                   0                   0                   0 
       State_Region       Meals_At_Home           ricepds_v          Wheatpds_q 
                  0                   0                   0                   0 
          chicken_q            pulsep_q           wheatos_q No_of_Meals_per_day 
                  0                   0                   0                   0 


In [24]:
%%R
# Impute missing values with mean for specific columns
impute_with_mean=function(column) {
  if (any(is.na(column))) {
    column[is.na(column)] =mean(column, na.rm = TRUE)
  }
  return(column)
}
gujnew$Meals_At_Home <- impute_with_mean(gujnew$Meals_At_Home)

In [25]:
%%R
# Check for missing values after imputation
cat("Missing Values After Imputation:\n")
print(colSums(is.na(gujnew)))


Missing Values After Imputation:
            state_1            District              Region              Sector 
                  0                   0                   0                   0 
       State_Region       Meals_At_Home           ricepds_v          Wheatpds_q 
                  0                   0                   0                   0 
          chicken_q            pulsep_q           wheatos_q No_of_Meals_per_day 
                  0                   0                   0                   0 


In [20]:
%%R
# Finding outliers and removing them
remove_outliers=function(df, column_name) {
  Q1 =quantile(df[[column_name]], 0.25)
  Q3= quantile(df[[column_name]], 0.75)
  IQR= Q3 - Q1
  lower_threshold= Q1 - (1.5 * IQR)
  upper_threshold=Q3 + (1.5 * IQR)
  df=subset(df, df[[column_name]] >= lower_threshold & df[[column_name]] <= upper_threshold)
  return(df)
}

In [26]:
%%R
outlier_columns= c("ricepds_v", "chicken_q")
for (col in outlier_columns) {
  gujnew =remove_outliers(gujnew, col)
}

In [28]:
%%R
# Summarize consumption
gujnew$total_consumption = rowSums(gujnew[, c("ricepds_v", "Wheatpds_q", "chicken_q", "pulsep_q", "wheatos_q")], na.rm = TRUE)

# Summarize and display top and bottom consuming districts and regions
summarize_consumption=function(group_col) {
  summary= gujnew %>%
    group_by(across(all_of(group_col))) %>%
    summarise(total = sum(total_consumption)) %>%
    arrange(desc(total))
  return(summary)
}

In [30]:
%%R
district_summary=summarize_consumption("District")
region_summary= summarize_consumption("Region")

In [31]:
%%R
cat("Top 3 Consuming Districts:\n")
print(head(district_summary, 3))
cat("Bottom 3 Consuming Districts:\n")
print(tail(district_summary, 3))

cat("Region Consumption Summary:\n")
print(region_summary)

Top 3 Consuming Districts:
# A tibble: 3 × 2
  District total
     <int> <dbl>
1        7 1392.
2        9  857.
3       22  750.
Bottom 3 Consuming Districts:
# A tibble: 3 × 2
  District total
     <int> <dbl>
1        5  46.6
2       21  34.2
3       25  28.4
Region Consumption Summary:
# A tibble: 5 × 2
  Region total
   <int> <dbl>
1      5 2320.
2      2 1948.
3      1 1483.
4      3  314.
5      4  158.


In [35]:
%%R
# Rename districts and sectors , get codes from appendix of NSSO 68th ROund Data
district_mapping=c("1" = "Ahmedabad", "2"="Amreli","3"="Anand","4"="Aravalli","5"="Banaskantha",
                      "6"="Bharuch","7"="Bhavnagar","8"="Botad","9"="Chota Udaipur","10"="Dahid","11"="Dang",
                      "12"="Dwarka","13"="Gandhinagar","14"="Gor Somnath","15"="Jamnagar","16"="Junagadh",
                      "17"="Kutch","18"="Kheda","19"="Mahisagar","20"="Mehsana","21"="Morbi","22"="Narmada")
sector_mapping =c("2" = "URBAN", "1" = "RURAL")

NULL


In [ ]:
%%R
gujnew$District= as.character(gujnew$District)
gujnew$Sector =as.character(gujnew$Sector)
gujnew$District= ifelse(gujnew$District %in% names(district_mapping), district_mapping[gujnew$District],
                          gujnew$District)
gujnew$Sector= ifelse(gujnew$Sector %in% names(sector_mapping), sector_mapping[gujnew$Sector], gujnew$Sector)


In [ ]:
%%R
# Test for differences in mean consumption between urban and rural
rural=gujnew %>%
  filter(Sector == "RURAL") %>%
  select(total_consumption)

urban= gujnew %>%
  filter(Sector == "URBAN") %>%
  select(total_consumption)

mean_rural= mean(rural$total_consumption)
mean_urban= mean(urban$total_consumption)


In [ ]:
%%R
# Perform z-test
z_test_result <- z.test(rural, urban, alternative = "two.sided", mu = 0, sigma.x = 2.56, sigma.y = 2.34, conf.level = 0.95)

# Generate output based on p-value
if (z_test_result$p.value < 0.05) {
  cat(glue::glue("P value is < 0.05 i.e. {round(z_test_result$p.value,5)}, Therefore we reject the null hypothesis.\n"))
  cat(glue::glue("There is a difference between mean consumptions of urban and rural.\n"))
  cat(glue::glue("The mean consumption in Rural areas is {mean_rural} and in Urban areas its {mean_urban}\n"))
} else {
  cat(glue::glue("P value is >= 0.05 i.e. {round(z_test_result$p.value,5)}, Therefore we fail to reject the null hypothesis.\n"))
  cat(glue::glue("There is no significant difference between mean consumptions of urban and rural.\n"))
  cat(glue::glue("The mean consumption in Rural area is {mean_rural} and in Urban area its {mean_urban}\n"))
}
